In [2]:
# ffe_path = 'C:/Users/leeyj/lab_ws/data/radar/AH1/AH-1_PO_3D_PEC_1.ffe'

# def read_ffe_file(file_path):
#     try:
#         with open(file_path, 'r') as file:
#             contents = file.readlines()
#             return contents[:100]     # 처음 10줄을 반환
#     except Exception as e:
#         return str(e)               # 출력

# read_ffe_file(ffe_path)

['##File Type: Far field\n',
 '##File Format: 7\n',
 '##Source: AH-1_LE-PO_8GHz_3D\n',
 '##Date: 2024-05-26 11:10:45\n',
 '** File exported by POSTFEKO version 2019.3.3-373003 (x64)\n',
 '\n',
 '#Configuration Name: StandardConfiguration1\n',
 '#Request Name: FarField1\n',
 '#Frequency: 8.00000000E+9\n',
 '#Coordinate System: Spherical\n',
 '#No. of Theta Samples: 181\n',
 '#No. of Phi Samples: 361\n',
 '#Result Type: RCS\n',
 '#Incident Wave Direction: (0.00000000E+0, 0.00000000E+0)\n',
 '#No. of Header Lines: 1\n',
 '#                 "U"               "V"               "Re(Eu)"          "Im(Eu)"          "Re(Ev)"          "Im(Ev)"          "RCS(U)"          "RCS(V)"          "RCS(Total)"      \n',
 '                    0.00000000E+0     0.00000000E+0    -4.85110198E+1    -2.59280597E+1    -1.95383806E-1    -2.15064875E-1     3.80206013E+4     1.06095017E+0     3.80216623E+4   \n',
 '#Configuration Name: StandardConfiguration1\n',
 '#Request Name: FarField1\n',
 '#Frequency: 8.000000

In [1]:
# 경로는 필요에 맞게 알맞게 수정

ffe_file_path = 'C:/Users/leeyj/lab_ws/data/radar/AH1/AH-1_PO_3D_PEC_1.ffe'
output_csv_path = 'C:/Users/leeyj/lab_ws/data/radar/AH1/AH-1_PO_3D_PEC_1_Test2.csv'

def process_and_save_ffe_to_csv(ffe_file_path, output_csv_path):
    import csv
    import math
    
    def convert_to_dbsm(rcs_m2):
        """Convert RCS from square meters to decibels (dBsm)."""
        return 10 * math.log10(rcs_m2)
    
    with open(ffe_file_path, 'r') as file:
        lines = file.readlines()
    
    # RCS 값에 맞는 부분을 찾음
    data_start_index = None
    for i, line in enumerate(lines):
        if 'RCS(Total)' in line:
            data_start_index = i + 1
            break

    if data_start_index is None:
        return "Data section not found."

    # RCS 값을 dBsm으로 변경한 후, 맞는 고각과 방위각에 할당
    rcs_data = {}
    for line in lines[data_start_index:]:
        if line.strip() == '':
            break
        parts = line.split()
        try:
            elevation = int(float(parts[0]))  # U 고각
            azimuth = int(float(parts[1]))  # V 방위각
            rcs_value_m2 = float(parts[-1])  # m^2으로 표현된 RCS 값
            rcs_value_dbsm = convert_to_dbsm(rcs_value_m2)  # m^2 to dBsm
            if elevation not in rcs_data:
                rcs_data[elevation] = {}
            rcs_data[elevation][azimuth] = rcs_value_dbsm
        except ValueError:
            continue

    # csv 파일로 작성
    with open(output_csv_path, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['Elevation Angle (degrees)', 'Azimuth Angle (degrees)', 'RCS (dBsm)'])
        for el in range(181):  # Elevation angles from 0 to 180 degrees
            for az in range(361):  # Azimuth angles from 0 to 360 degrees
                rcs_value = rcs_data.get(el, {}).get(az, 'NaN')
                writer.writerow([el, az, rcs_value])

    return "CSV file created successfully."

process_and_save_ffe_to_csv(ffe_file_path, output_csv_path)

'CSV file created successfully.'